In [1]:
import numpy as np
import pandas as pd

In [2]:
%pip install -U gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 22.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Uninstalling gensim-4.0.1:
      Successfully uninstalled gensim-4.0.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import os
import gensim

In [4]:
data = []

df=pd.read_csv('/kaggle/input/devrev/train_data.csv')
for i in range(df.shape[0]):
    rw=df.iloc[i]
    data.append('Paragraph')

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [5]:
df['id']=list(i for i in range(df.shape[0]))
df.head()

,Unnamed: 0,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start,id
0,2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],[526],0
1,6,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],[505],1
2,7,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360],2
3,8,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],[276],3
4,9,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290],4


In [6]:
#FULL
max_epochs = 1
vec_size = 728
alpha = 0.025

model_full = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=70,dm=1,alpha = 0.025)
  
model_full.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model_full.train(tagged_data,
                total_examples=model_full.corpus_count,
                epochs=model_full.epochs)
    # decrease the learning rate
    model_full.alpha -= 0.0002
    # fix the learning rate, no decay
    model_full.min_alpha = model_full.alpha

model_full.save("d2v.model")
print("Model Saved")

iteration 0
Model Saved


In [7]:
from gensim.models.doc2vec import Doc2Vec

model_full= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model_full.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [-6.59395242e-04 -3.20572086e-04  4.08350694e-04 -2.78144667e-04
 -2.86153168e-04 -5.18208602e-04 -2.91162374e-04 -2.38278939e-04
  2.45034862e-05 -4.52721433e-05  3.77281831e-04  5.01612385e-06
 -5.32023958e-04  2.41687580e-04 -6.12406759e-04  5.17938985e-04
 -5.86889917e-04 -1.55910806e-04  6.48144283e-04 -1.31023990e-04
 -3.23347012e-05 -6.59853918e-04 -6.17475307e-04  6.75780408e-04
  6.69586356e-04  1.22422367e-04  2.90750468e-04  3.62527004e-04
  3.44352971e-04  5.62615227e-04  2.73574347e-04 -2.34684412e-04
  3.61755170e-04 -6.45121618e-05  2.36444510e-04 -3.71791757e-05
 -5.64177608e-05  5.92060154e-04 -5.65314840e-05  3.59754398e-04
 -5.21973125e-04 -2.57646549e-04  3.25425877e-04  1.83583950e-04
  4.30588203e-04 -5.38295950e-04 -2.59253593e-05 -1.39121621e-04
 -5.82587090e-04  3.44180560e-04  2.63175461e-04  6.53081806e-04
  4.24122758e-04  3.48552654e-04 -6.82640064e-04 -7.26741928e-05
  6.24094857e-04  2.67156138e-04  2.72353995e-04 -2.33694707e-04
  1.48324354e-04

In [8]:
#Pretrained
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))
model_pre = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=40,dm=1)
model_pre.build_vocab(train_corpus)
model_pre.train(train_corpus, total_examples=model_pre.corpus_count, epochs=model_pre.epochs)



In [ ]:
dict_vals={}
df=pd.read_csv('/kaggle/input/devrev/train_data.csv')
for i in range(df.shape[0]):
    dict_vals[df.iloc[i]['Theme']]=[]
    
for i in range(df.shape[0]):
    dict_vals[df.iloc[i]['Theme']].append(df.iloc[i]['Paragraph'])
dict_models={}
for i in dict_vals.keys():
    data=dict_vals[i]
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    model_semi = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=70,dm=1,alpha = 0.025)
  
    model_semi.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch),"Theme = ",i)
        model_semi.train(tagged_data,
                    total_examples=model_semi.corpus_count,
                    epochs=model_semi.epochs)
        # decrease the learning rate
        model_semi.alpha -= 0.0002
        # fix the learning rate, no decay
        model_semi.min_alpha = model_semi.alpha

    model_semi.save(i+"-d2v.model")
    dict_models[i]=model_semi
    print("Model Saved")
    

iteration 0 Theme =  Beyoncé
Model Saved
iteration 0 Theme =  Frédéric_Chopin
Model Saved
iteration 0 Theme =  Sino-Tibetan_relations_during_the_Ming_dynasty
Model Saved
iteration 0 Theme =  The_Legend_of_Zelda:_Twilight_Princess
Model Saved
iteration 0 Theme =  Spectre_(2015_film)
Model Saved
iteration 0 Theme =  New_York_City
Model Saved
iteration 0 Theme =  To_Kill_a_Mockingbird
Model Saved
iteration 0 Theme =  Solar_energy
Model Saved
iteration 0 Theme =  Kanye_West
Model Saved
iteration 0 Theme =  Buddhism
Model Saved
iteration 0 Theme =  American_Idol
Model Saved
iteration 0 Theme =  Dog
Model Saved
iteration 0 Theme =  2008_Summer_Olympics_torch_relay
Model Saved
iteration 0 Theme =  Genome
Model Saved
iteration 0 Theme =  Comprehensive_school
Model Saved
iteration 0 Theme =  Republic_of_the_Congo
Model Saved
iteration 0 Theme =  Prime_minister
Model Saved
iteration 0 Theme =  Institute_of_technology
Model Saved
iteration 0 Theme =  Dutch_Republic
Model Saved
iteration 0 Theme =

In [ ]:
from scipy import spatial
def get_thresh():
    thresh_pre,thresh_full=[0,0,0,0],[0,0,0,0]
    dict_thresh_semi={}
    for i in dict_vals.keys():
        dict_thresh_semi[i]=[0,0,0,0]
    
    for i in range(df.shape[0]):
        rw=df.iloc[i]
        if i%1000==0:
            print("At i = ",i)
        context=rw['Paragraph']
        ques=rw['Question']
        test_context = word_tokenize(context.lower())
        test_ques = word_tokenize(ques.lower())
        
        v1_full = model_full.infer_vector(test_context)
        v2_full = model_full.infer_vector(test_ques)
        
        v1_pre = model_pre.infer_vector(test_context)
        v2_pre = model_pre.infer_vector(test_ques)
        
        model_sm=dict_models[rw['Theme']]
        
        v1_semi = model_sm.infer_vector(test_context)
        v2_semi = model_sm.infer_vector(test_ques)
        
        result_full = 1 - spatial.distance.cosine(v1_full, v2_full)
        result_pre = 1 - spatial.distance.cosine(v1_pre, v2_pre)
        result_semi = 1 - spatial.distance.cosine(v1_semi, v2_semi)
        
        if rw['Answer_possible']==True:
            thresh_pre[0]+=result_pre
            thresh_full[0]+=result_full
            dict_thresh_semi[rw['Theme']][0]+=result_semi
            dict_thresh_semi[rw['Theme']][2]+=1
            thresh_full[2]+=1
            thresh_pre[2]+=1
            
        else :
            thresh_pre[1]+=result_pre
            thresh_full[1]+=result_full
            dict_thresh_semi[rw['Theme']][1]+=result_semi
            dict_thresh_semi[rw['Theme']][3]+=1
            thresh_full[3]+=1
            thresh_pre[3]+=1
    
    return thresh_pre,thresh_full,dict_thresh_semi
    

In [ ]:
r1,r2,r3=get_thresh()

In [ ]:
thresh_pre=r1[0]/r1[2]
thresh_full=r2[0]/r2[2]
thresh_semi={}
for i in dict_vals.keys():
    thresh_semi[i]=r3[i][0]/r3[i][2]

print("Thresh for pretrained model correct context-question similarity = ",thresh_pre)
print("Thresh for full trained model correct context-question similarity = ",thresh_full)


In [ ]:
thresh_answer_full=r2[0]/r2[2]
thresh_unanswer_full=r2[1]/r2[3]
thresh_answer_pre=r1[0]/r1[2]
thresh_unanswer_pre=r1[1]/r1[3]
dict_thresh_semi={}
for i in dict_vals.keys():
        dict_thresh_semi[i]=[0,0]
for i in r3.keys():
    dict_thresh_semi[i][0]=r3[i][0]/r3[i][2]
    dict_thresh_semi[i][1]=r3[i][1]/r3[i][3]


In [ ]:
dict_context_embed={}
for i in range(df.shape[0]):
    dict_context_embed[i]=[]
for i in range(df.shape[0]):
    rw=df.iloc[i]
    if (i)%1000==0:
        print("At i = ",i)
    context=rw['Paragraph']
    test_context = word_tokenize(context.lower())
    v1_full = model_full.infer_vector(test_context)
    v1_pre = model_pre.infer_vector(test_context)
#     model_sm=dict_models[rw['Theme']]
#     v1_semi = model_sm.infer_vector(test_context)
    dict_context_embed[i]=[v1_pre,v1_full]
    
    

In [ ]:
df.head()

In [12]:
sample_context=[]
sample_ques="None"
sample_theme="None"
sample_ids=[]
for i in range(5):
    rw=df.iloc[i]
    sample_context.append(rw['Paragraph'])
    sample_ques=rw['Question']
    sample_theme=rw['Theme']
    sample_ids.append(i)
    

In [17]:

encoded_inputs = tokenizer(
    questions=["What is love ?","What is love ?","What is love ?"],
    titles=["Haddaway","Haddaway","Haddaway"],
    texts=["'What Is Love' is a song recorded by the artist Haddaway","'What Is Love' is a song recorded by the artist Haddaway","'What Is Love' is a song recorded by the artist Haddaway"],
    return_tensors="tf",
)
outputs = model(encoded_inputs)
start_logits = outputs.start_logits
end_logits = outputs.end_logits
relevance_logits = outputs.relevance_logits

In [32]:
start_indices=np.argmax(start_logits.numpy(),axis=1).tolist()
end_indices=np.argmax(end_logits.numpy(),axis=1).tolist()
best_ans=np.argmax(relevance_logits.numpy())

In [40]:
tokenizer.decode(encoded_inputs['input_ids'][best_ans][start_indices[best_ans]:end_indices[best_ans]+1])

'a song'

In [ ]:
from transformers import TFDPRReader, DPRReaderTokenizer

tokenizer = DPRReaderTokenizer.from_pretrained("facebook/dpr-reader-single-nq-base")
model = TFDPRReader.from_pretrained("facebook/dpr-reader-single-nq-base", from_pt=True)

In [48]:

def get_reader_answer(ques,retriever_contexts,tm,method='dpr'):
    if method=='dpr':
        question=[]
        theme=[]
        for i in range(len(retriever_contexts)):
            question.append(ques)
            theme.append(tm)
        encoded_inputs = tokenizer(
            questions=question,
            titles=theme,
            texts=retriever_contexts,
            return_tensors="tf",
            padding=True,
            truncation=True
        )
        outputs = model(encoded_inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits
        relevance_logits = outputs.relevance_logits

        start_indices=np.argmax(start_logits.numpy(),axis=1).tolist()
        end_indices=np.argmax(end_logits.numpy(),axis=1).tolist()
        best_ans=np.argmax(relevance_logits.numpy())

    #     print("answers are ")
    #     for i in range(len(start_indices)):
    #         print("Score = ",relevance_logits.numpy()[i])
    #         print("Answer = ",tokenizer.decode(encoded_inputs['input_ids'][i][start_indices[i]:end_indices[i]+1]))

        return tokenizer.decode(encoded_inputs['input_ids'][best_ans][start_indices[best_ans]:end_indices[best_ans]+1]),relevance_logits.numpy()[best_ans]
    
    elif method=='roberta':
        return None

In [49]:
v1,v2=get_reader_answer(sample_ques,sample_context,sample_theme,'dpr')

In [50]:
v1

'in the late 1990s as lead singer of r & b'

In [59]:
from scipy import spatial

In [64]:
def predict(context,questions,context_ids,model_type='full',theme=None,top_k=5,thresh=0.40):
    sim_scores=[]
    context_embed=0
    question_embed=0
    if model_type=='full':
        context_embed = model_full.infer_vector(context)
    elif model_type=='pre':
        context_embed = model_pre.infer_vector(context)
    else :
        model_sm=dict_models[theme]
        context_embed = model_sm.infer_vector(context)
    questions_embed=[]
    for i in range(len(questions)):
        result1=0
        if model_type=='full':
            question_embed = dict_context_embed[context_ids[i]][1]
            result1 = 1 - spatial.distance.cosine(context_embed, question_embed)
        elif model_type=='pre':
            question_embed = dict_context_embed[context_ids[i]][0]
            result1 = 1 - spatial.distance.cosine(context_embed, question_embed)
        else :
            question_embed = dict_context_embed[context_ids[i]][2]
            result1 = 1 - spatial.distance.cosine(context_embed, question_embed)    
        questions_embed.append(question_embed)
        sim_scores.append(result1)
        
    array = np.array(sim_scores)
    indices = np.argsort(array)
    
    print("Got similarities score as ",array)
    
    retriever_contexts=[]
    for i in range(min(top_k,len(indices))):
        if array[indices[i]]>=thresh:
            retriever_contexts.append(questions[indices[i]])
            print("Using context ",questions[indices[i]])
            
    answer=""
    socre=1
    if len(retriever_contexts)>0:
        answer,socre=get_reader_answer(context,retriever_contexts,theme,method='dpr')
        
    return answer,socre
    
    
    
    

In [68]:
v1,v2=predict(list(sample_ques),sample_context,sample_ids,'pre',sample_theme,3,0.20)

Got similarities score as  [ 0.00314397 -0.00081992 -0.00377745  0.00249629 -0.00714049]


In [69]:
v1

1